In [1]:
%matplotlib inline

In [2]:
from collections import defaultdict
from functools import partial
import math
import re

from matplotlib import pyplot as plt
import numexpr
import numpy as np
import pandas as pd

In [3]:
N_PARAMS = 8
N_DESC_COLS = 6
N_DSC2_COLS = 4
N_DSC3_COLS = 7

LVLDAM_BREAKPOINTS = [(0, 1), (1, 8), (8, 16), (16, 22), (22, 28), (28, None)]

# Read skills data
Extracted from patch_d2.mpq

In [4]:
skills = pd.read_csv('data/skills.txt', delimiter='\t')
skilldesc = pd.read_csv('data/skilldesc.txt', delimiter='\t')

## Combine strings/elem tables
Extracted from respective mpq files ENG localisation.

In [5]:
strings = pd.read_csv('data/string.csv', header=0, names=('key', 'value'))
expansionstring = pd.read_csv('data/expansionstring.csv', header=0, names=('key', 'value'))
patchstring = pd.read_csv('data/patchstring.csv', header=0, names=('key', 'value'))

In [6]:
all_strings = (
    pd.concat([
        strings.assign(source='strings'),
        expansionstring.assign(source='expansionstring'),
        patchstring.assign(source='patchstring'),
    ])
    .drop_duplicates(subset=['key'], keep='last')
)

In [7]:
all_strings.source.value_counts()

strings            4919
expansionstring    2700
patchstring        1061
Name: source, dtype: int64

### Prepare mappings from strings tables

In [8]:
def get_string_mapped_columns():
    string_mapped_columns = ['str name', 'str long', 'str alt', 'str mana']
    for col_root, line_limit in {'desctext': N_DESC_COLS, 'dsc2text': N_DSC2_COLS, 'dsc3text': N_DSC3_COLS}.items():
        for a_b in 'ab':
            for i in range(1, line_limit + 1):
                string_mapped_columns.append(f'{col_root}{a_b}{i}')
    return string_mapped_columns

string_mapped_columns = get_string_mapped_columns()

In [9]:
def clean_string(s):
    try:
        cleaned = s.replace('\\n', '\n')
        if s[:2].isnumeric():
            return cleaned
        return '\n'.join(reversed(cleaned.split('\n')))  # reverse skill strings
    except AttributeError:
        return s
    

strings_map = (
    all_strings.set_index('key').value
    .apply(clean_string)
    .to_dict()
)

### Prepare mappings for EType

In [10]:
elem_types = pd.read_csv('data/ElemTypes.txt', delimiter='\t')
elemental_type_map = elem_types.dropna().set_index('Code')['Elemental Type'].to_dict()

# Merge descriptions for character skills

In [11]:
charskills = (
    skills
    .loc[skills.charclass.notnull()]
    .merge(skilldesc, how='left', on='skilldesc', validate='one_to_one')
)
# Replace string keys with values from .tbl files
charskills.loc[:, string_mapped_columns] = charskills[string_mapped_columns].replace(strings_map)
# Replace elemental types with mapped values
charskills.loc[:, 'EType'] = charskills.EType.replace(elemental_type_map)

# Create lookup

In [12]:
def get_skill_details(charskills):
    skill_details = {}
    for index, row in charskills.iterrows():
        row = row.copy().replace({np.nan: None})
        row_details = _get_skill_details_for_row(row)
        
        skill_key = title_to_camelcase(row.skill)
        skill_details[skill_key] = {k: v for k, v in row_details.items() if v}
    return skill_details

def _get_skill_details_for_row(row):
    return {
        'strName': row['str name'],
        'strLong': row['str long'],
        'skillPage': row.SkillPage,
        'skillRow': row.SkillRow,
        'skillColumn': row.SkillColumn,
        'strMana': row['str mana'],
        'mana': safe_int(row['mana']),
        'lvlMana': safe_int(row['lvlmana']),
        'minMana': safe_int(row['minmana']),
        'manaShift': safe_int(row['manashift']),
        'toHit': safe_int(row.ToHit),
        'levToHit': safe_int(row.LevToHit),
        'toHitCalc': row.ToHitCalc,
        'hitShift': safe_int(row.HitShift),
        'srcDam': safe_int(row.SrcDam),
        'minDam': safe_int(row.MinDam),
        'minLevDam1': safe_int(row.MinLevDam1),
        'minLevDam2': safe_int(row.MinLevDam2),
        'minLevDam3': safe_int(row.MinLevDam3),
        'minLevDam4': safe_int(row.MinLevDam4),
        'minLevDam5': safe_int(row.MinLevDam5),
        'maxDam': safe_int(row.MaxDam),
        'maxLevDam1': safe_int(row.MaxLevDam1),
        'maxLevDam2': safe_int(row.MaxLevDam2),
        'maxLevDam3': safe_int(row.MaxLevDam3),
        'maxLevDam4': safe_int(row.MaxLevDam4),
        'maxLevDam5': safe_int(row.MaxLevDam5),
        'dmgSymPerCalc': row.DmgSymPerCalc,
        'eType': row.EType,
        'eMin': safe_int(row.EMin),
        'eMinLev1': safe_int(row.EMinLev1),
        'eMinLev2': safe_int(row.EMinLev2),
        'eMinLev3': safe_int(row.EMinLev3),
        'eMinLev4': safe_int(row.EMinLev4),
        'eMinLev5': safe_int(row.EMinLev5),
        'eMax': safe_int(row.EMax),
        'eMaxLev1': safe_int(row.EMaxLev1),
        'eMaxLev2': safe_int(row.EMaxLev2),
        'eMaxLev3': safe_int(row.EMaxLev3),
        'eMaxLev4': safe_int(row.EMaxLev4),
        'eMaxLev5': safe_int(row.EMaxLev5),
        'eDmgSymPerCalc': row.EDmgSymPerCalc,
        'params': {f'par{i}': int(row[f'Param{i}']) for i in range(1, N_PARAMS + 1) if row[f'Param{i}']},
        'descLines': _get_desclines_for_row(row, 'desc', N_DESC_COLS),
        'dsc2Lines': _get_desclines_for_row(row, 'dsc2', N_DSC2_COLS),
        'dsc3Lines': _get_desclines_for_row(row, 'dsc3', N_DSC3_COLS),
    }
    
def _get_desclines_for_row(row, column_root, max_entries):
    row = row.copy().replace({np.nan: None})
    
    entries = []
    for i in range(1, max_entries + 1):
        if not row[f'{column_root}line{i}']:
            continue

        entry = {
            f'{column_root}Line': int(row[f'{column_root}line{i}']),
            f'{column_root}TextA': row[f'{column_root}texta{i}'],
            f'{column_root}TextB': row[f'{column_root}textb{i}'],
            f'{column_root}CalcA': safe_int(row[f'{column_root}calca{i}']),
            f'{column_root}CalcB': safe_int(row[f'{column_root}calcb{i}']),
        }
        entries.append({k: v for k, v in entry.items() if v})
        
    if column_root == 'desc':
        entries.reverse()  # D2 renders desclines bottom-up
    return entries

def title_to_camelcase(s):
    first_lowered = s[:1].lower() + s[1:] if s else ''
    return first_lowered.replace(' ', '').replace('_', '')

def safe_int(x):
    try:
        return int(x)
    except (TypeError, ValueError):
        return x

In [13]:
skill_details = get_skill_details(charskills)

# Generate descriptions from lookup

## Look up skills by descline for development

In [14]:
def select_skills_with_descline(charskills, pattern):
    descline_columns = get_descline_columns()
    return charskills.loc[(charskills[descline_columns] == pattern).any(axis=1), ['skill'] + descline_columns]

def get_descline_columns():
    descline_columns = []
    for col_root, line_limit in {'desc': N_DESC_COLS, 'dsc2': N_DSC2_COLS, 'dsc3': N_DSC3_COLS}.items():
        for i in range(1, line_limit + 1):
            descline_columns.append(f'{col_root}line{i}')
    return descline_columns

In [15]:
select_skills_with_descline(charskills, 7)

,skill,descline1,descline2,descline3,descline4,descline5,descline6,dsc2line1,dsc2line2,dsc2line3,dsc2line4,dsc3line1,dsc3line2,dsc3line3,dsc3line4,dsc3line5,dsc3line6,dsc3line7
3,Critical Strike,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Multiple Shot,1.0,7.0,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Dodge,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Avoid,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Evade,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Pierce,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Lightning Strike,1.0,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,63.0,63.0,63.0,63.0,NaN,NaN
31,Warmth,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,Charged Bolt,1.0,7.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,63.0,NaN,NaN,NaN,NaN,NaN
47,Chain Lightning,1.0,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,63.0,63.0,63.0,NaN,NaN,NaN


## Define calculators

In [16]:
def calculate_mana_cost(skill, skill_levels, lvl, precision=10):
    mana, lvl_mana, mana_shift = skill['mana'], skill.get('lvlMana', 0), skill['manaShift']
    cost = (precision * (mana + lvl_mana * (lvl - 1)) << mana_shift >> 8) / precision
    return max(cost, skill.get('minMana', 0))

def calculate_to_hit(skill, skill_levels, lvl):
    to_hit_expression = skill.get('toHitCalc')
    if to_hit_expression:
        return calculate_skill_value(to_hit_expression, skill, skill_levels, lvl)

    return skill['toHit'] + skill['levToHit'] * (lvl - 1)

def calculate_damage(skill, skill_levels, lvl, initial_damage_key, damage_per_level_key_root):
    damage_per_level_values = (
        [skill[initial_damage_key]] + [skill[f'{damage_per_level_key_root}{i}'] for i in range(1, 6)]
    )
    damage = 0
    for (lower, upper), damage_per_level in zip(LVLDAM_BREAKPOINTS, damage_per_level_values):
        if lvl <= lower:
            break
        lvl_for_band = min(upper, lvl) if upper is not None else lvl
        damage += (lvl_for_band - lower) * damage_per_level

    return int(damage) << skill['hitShift'] >> 8

def create_linear_calculator(param_key_a, param_key_b):
    def calculator(skill, skill_levels, lvl):
        a = skill['params'].get(param_key_a, 0)
        b = skill['params'].get(param_key_b, 0)
        return a + b * (lvl - 1)
    return calculator

def create_diminishing_calculator(param_key_a, param_key_b):
    def calculator(skill, skill_levels, lvl):
        a = skill['params'].get(param_key_a, 0)
        b = skill['params'].get(param_key_b, 0)
        return a + ((110 * lvl) * (b - a)) / (100 * (lvl + 6))
    return calculator

def _get_param(skill, skill_levels, lvl, param_number):
    return skill['params'][f'par{param_number}']

def _get_param_xlvl(skill, skill_levels, lvl, param_number):
    return _get_param(skill, skill_levels, lvl, param_number) * (lvl - 1)

def calculate_skill_value(calc_expression, skill, skill_levels, lvl):
    if not calc_expression:
        return None
    
    skills_filled = re.sub(
        pattern=r"skill\('((?:\w|\s)+)'.blvl\)",
        repl=partial(_matched_skill_level, skill_levels=skill_levels),
        string=str(calc_expression),
    )
    skills_and_calcs_filled = re.sub(
        pattern='|'.join(CALC_LOOKUP.keys()),
        repl=partial(_matched_calc, skill=skill, skill_levels=skill_levels, lvl=lvl),
        string=skills_filled,
    )
    return numexpr.evaluate(skills_and_calcs_filled).item()

def _matched_skill_level(match, skill_levels, group=1):
    skill_name = title_to_camelcase(match[group])
    return str(skill_levels.get(skill_name, 0))

def _matched_calc(match, skill, skill_levels, lvl, group=0):
    calc_key = match[group]
    calculator = CALC_LOOKUP[calc_key]
    return str(calculator(skill, skill_levels, lvl))

In [17]:
CALC_LOOKUP = {
    'toht': calculate_to_hit,
    'edmn': partial(calculate_damage, initial_damage_key='eMin', damage_per_level_key_root='eMinLev'),
    'edmx': partial(calculate_damage, initial_damage_key='eMax', damage_per_level_key_root='eMaxLev'),
    'ln12': create_linear_calculator('par1', 'par2'),
    'ln34': create_linear_calculator('par3', 'par4'),
    'ln56': create_linear_calculator('par5', 'par6'),
    'ln78': create_linear_calculator('par7', 'par8'),
    'dm12': create_diminishing_calculator('par1', 'par2'),
    **{f'par{i}': partial(_get_param, param_number=i) for i in range(1, 9)},
}

## Define formatters

In [18]:
FRAMES_PER_SECOND = 25
YARDS_PER_GAME_UNIT = 2 / 3

def format_mana_cost(skill, skill_levels, lvl, ta, tb, ca, cb):
    cost = calculate_mana_cost(skill, skill_levels, lvl, precision=ca or 10)
    if not cost % 1:  # remove decimal places if not used
        cost = int(cost)
    return f"{skill['strMana']}{cost}"

def format_attack_rating(skill, skill_levels, lvl, ta, tb, ca, cb):
    return f"Attack: +{calculate_to_hit(skill, skill_levels, lvl)} percent"

def format_physical_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('minDam') is None:
        return ''
    
    synergy_multiplier = calculate_skill_value(skill.get('dmgSymPerCalc'), skill, skill_levels, lvl) or 1
    synergy_multiplier = (100 + synergy_bonus) / 100

    min_damage = synergy_multiplier * calculate_damage(skill, skill_levels, lvl, 'minDam', 'minLevDam')
    max_damage = synergy_multiplier * calculate_damage(skill, skill_levels, lvl, 'maxDam', 'maxLevDam')

    if min_damage == max_damage:
        return f'Damage: +{min_damage}'
    
    return f'Damage: +{min_damage}-{max_damage}'

def format_elemental_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('eMin') is None:
        return ''
    
    synergy_bonus = calculate_skill_value(skill.get('eDmgSymPerCalc'), skill, skill_levels, lvl) or 0
    synergy_multiplier = (100 + synergy_bonus) / 100

    min_damage = math.floor(synergy_multiplier * calculate_damage(skill, skill_levels, lvl, 'eMin', 'eMinLev'))
    max_damage = math.floor(synergy_multiplier * calculate_damage(skill, skill_levels, lvl, 'eMax', 'eMaxLev'))

    if min_damage == max_damage:
        return f"{skill['eType']} Damage: +{min_damage}"
    
    return f"{skill['eType']} Damage: +{min_damage}-{max_damage}"

def replace_ta_with_calc_on_ca(skill, skill_levels, lvl, ta, tb, ca, cb, pattern='%d%'):
    return ta.replace(pattern, str(calculate_skill_value(ca, skill, skill_levels, lvl)))

def create_calc_formatter_on_ca(template, frames=False, game_units=False, precision=10):
    def formatter(skill, skill_levels, lvl, ta, tb, ca, cb):
        calc = calculate_skill_value(ca, skill, skill_levels, lvl)
        if frames:
            calc /= FRAMES_PER_SECOND
        if game_units:
            calc *= YARDS_PER_GAME_UNIT

        calc = math.floor(precision * calc) / precision
        if not calc % 1:  # remove decimal places if not used
            calc = int(calc)
        return template.format(calc=calc, ta=ta, tb=tb, ca=ca, cb=cb) if calc else ''
    return formatter

In [25]:
FORMATTERS_BY_DESCLINE = {
    1: format_mana_cost,
    2: create_calc_formatter_on_ca('{ta} {calc:+d}{tb}'),
    3: create_calc_formatter_on_ca('{ta}{calc}{tb}'),
    4: create_calc_formatter_on_ca('{ta}+{calc}'),
    5: create_calc_formatter_on_ca('{ta} {calc}'),
    6: create_calc_formatter_on_ca('+{calc} {ta}'),
    7: create_calc_formatter_on_ca('{calc} {ta}', precision=1),
    8: format_attack_rating,
    9: format_physical_damage,
    10: format_elemental_damage,
    11: lambda skill, skill_levels, lvl, ta, tb, ca, cb: None,  # figure out what this does
    12: create_calc_formatter_on_ca('{ta}{calc} seconds', frames=True),
    13: create_calc_formatter_on_ca('Life: {calc}'),  # results much lower than game screen, check necro pet calculator

    19: create_calc_formatter_on_ca('{ta}{tb} {calc} yards', game_units=True, precision=10),

    25: lambda skill, skill_levels, lvl, ta, tb, ca, cb: f"{ta or ''}{tb or ''}",

    40: lambda skill, skill_levels, lvl, ta, tb, ca, cb: ta.replace('%s', tb),  # ca refers to color, ignored here

    63: create_calc_formatter_on_ca('{ta}: +{calc}% {tb}'),

    66: replace_ta_with_calc_on_ca,
    67: create_calc_formatter_on_ca('{ta}: +{calc} {tb}', precision=1),

}

def format_descline(descline_number, skill, skill_levels, lvl, ta, tb, ca, cb):
    formatter = FORMATTERS_BY_DESCLINE[descline_number]
    return formatter(skill, skill_levels, lvl, ta, tb, ca, cb)

## Format skills

In [26]:
def format_skill(skill, skill_levels, lvl):
    lines = [
        skill['strName'],
        skill['strLong'],
        '',
        format_desclines('dsc2', skill, skill_levels, lvl),
        f'Current Level: {lvl}',
        format_desclines('desc', skill, skill_levels, lvl),
        f'Next Level: {lvl + 1}',
        format_desclines('desc', skill, skill_levels, lvl + 1),
        format_desclines('dsc3', skill, skill_levels, lvl),
    ]
    return '\n'.join([line for line in lines if line is not None])

def format_desclines(root, skill, skill_levels, lvl):
    lines = []
    for entry in skill.get(f'{root}Lines', []):
        line = format_descline(
            descline_number=entry[f'{root}Line'],
            skill=skill,
            skill_levels=skill_levels,
            lvl=lvl,
            ta=entry.get(f'{root}TextA'),
            tb=entry.get(f'{root}TextB'),
            ca=entry.get(f'{root}CalcA'),
            cb=entry.get(f'{root}CalcB'),
        )
        if line:
            lines.append(line)
    return '\n'.join(lines) + '\n' if lines else None

print(format_skill(skill_details['criticalStrike'], {'dopplezon': 4, 'explodingArrow': 10}, 10))

Critical Strike
passive - your attacks have a chance to do double damage

Current Level: 10
56  percent chance

Next Level: 11
58  percent chance



In [21]:
skill_details['criticalStrike']

{'strName': 'Critical Strike',
 'strLong': 'passive - your attacks have a chance to do double damage',
 'skillPage': 2,
 'skillRow': 1,
 'skillColumn': 3,
 'strMana': 'Mana Cost: ',
 'manaShift': 8,
 'hitShift': 8,
 'params': {'par1': 5, 'par2': 80},
 'descLines': [{'descLine': 7,
   'descTextA': ' percent chance',
   'descCalcA': 'dm12'}]}

# Deprecated synergy code

In [22]:
# skill_details['physicalSynergies'] = _get_synergies(row.DmgSymPerCalc, skill_details['params'])
# skill_details['elementalSynergies'] = _get_synergies(row.EDmgSymPerCalc, skill_details['params'])

def _get_synergies(calc_string, params):
    if not calc_string:
        return []
    
    weights = re.findall(r'(?:\s?\*\s?)(par\d)', calc_string)
    if not weights:
        raise ValueError('Given synergy calc_string with no parameters.')
    if len(weights) == 1:
        skill_names = re.findall(r"skill\('((?:\w|\s)+)'.blvl\)", calc_string)
        return [
            {'skill': title_to_camelcase(s), 'weight': params[weights[0]]}
            for s in skill_names
        ]
    else:
        matches = re.findall(r"skill\('((?:\w|\s)+)'.blvl\)(?:\s?\*\s?)(par\d)", calc_string)
        return [
            {'skill': title_to_camelcase(s), 'weight': params[w]}
            for (s, w) in matches
        ]
    
    return synergies

def _get_synergy_multiplier_precalc(synergies, skill_levels):  # uses pre-calculated synergy values
    if not synergies:
        return 1
    
    synergy_bonus = 0
    for synergy in synergies:
        synergy_level = skill_levels.get(synergy['skill'], 0)
        synergy_bonus += synergy['weight'] * synergy_level
    
    return (100 + synergy_bonus) / 100